# XOR neuronales Netz zu Testen
die Gewichte werden zufällig bestimmt 

In [1]:
import numpy as np

def tanh(x):
    return np.tanh(x)

def tanh_abl(x):
    return 1.0 - np.tanh(x)**2

def sig(x):
    return 1/(1 + np.exp(-x))

def sig_abl(x):
    return sig(x)*(1-sig(x))

In [2]:
class NeuralNetwork:
    def __init__(self, inp,hidd,out, act='tanh',bias=True):
        self.bias=bias
        self.actf=act
        self.inp=inp
        self.hidd=hidd
        self.out=out
        if bias:
            add=1
        else:
            add=0
        if act == 'sig':
            self.act = sig
            self.act_abl = sig_abl
        elif act == 'tanh':
            self.act = tanh
            self.act_abl = tanh_abl
        self.wih=2*np.random.random((hidd+add,inp+add))-1        
        self.who=2*np.random.random((out,hidd+add))-1
        self.fehler=0
        
           
    def prepareInput(self,inListe):
        #fuegt Bias zur einListe
        eingabe=np.array(inListe)
        if not self.bias:return eingabe
        X        = np.ones((len(eingabe),len(eingabe[0])+1))
        X[:,1:] = eingabe        
        return X
    
    def predict(self, x):
        hidden_inputs = np.dot(self.wih,x)
        hidden_outputs = self.act(hidden_inputs)
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.act(final_inputs)
        return final_outputs
        
    def fit(self,X,y,epochen,lernrate=0.1):
        for n in range(epochen):
            i=np.random.randint(0,X.shape[0])
            #Zufaellige Auswahl aus den Trainingsdaten
            #Festlegen der Shapes, die sind sonst nicht eindeutig
            xin=np.reshape(X[i],(X.shape[1],1))            
            targets=np.reshape(y[i],(y[i].shape[0],1))        
            hidden_inputs = np.dot(self.wih,xin)            
            hidden_outputs = self.act(hidden_inputs)            
            # calculate signals into final output layer
            final_inputs = np.dot(self.who, hidden_outputs)            
            final_outputs = self.act(final_inputs)            
            output_errors = targets - final_outputs
            self.fehler+=np.sum(output_errors**2)            
            hidden_errors = np.dot(self.who.T, output_errors)             
            if self.actf=="tanh":
                self.who += lernrate * output_errors *np.dot((1.0 - final_outputs**2),hidden_outputs.T  )  
                self.wih += lernrate * hidden_errors *np.dot((1.0 - hidden_outputs**2),xin.T)           
            else:                
                self.who += lernrate * np.dot( output_errors * final_outputs * (1.0 - final_outputs),hidden_outputs.T  )  
                self.wih += lernrate * np.dot(( hidden_outputs * (1.0 - hidden_outputs)),xin.T)
        print("Gewichte nach Runde:",n+1,"\nWHI\n",self.wih,"\nWHO\n",self.who)

 

In [5]:

#np.random.seed(0)#l
inListe=[[0, 0], [0, 1], [1, 0], [1, 1]]
ausListe=[0, 1, 1, 0]
for i in range(len(ausListe)):
    ausListe[i]=[ausListe[i]]
#inListe=[[0,0,0],[0,0,1],[0,1,0],[0,1,1],[1,0,0],[1,0,1],[1,1,0],[1,1,1]]
#ausListe=[0, 1, 1, 0,1,0,0,1]
y = np.array(ausListe)
inp=len(inListe[0])   
nn = NeuralNetwork(inp,2,1,"tanh",True)
X=nn.prepareInput(inListe)
#print("X\n",X,"\ny\n",y)
#print("WIH\n",nn.wih)
#print("WHO\n",nn.who)
nn.fit(X,y,10000,0.1)
for i in range(len(X)):
    print(X[i][1:],nn.predict(X[i]),y[i])
print("Fehler: ",nn.fehler)


Gewichte nach Runde: 10000 
WHI
 [[-0.80315708  3.2882235   2.31398481]
 [-0.3763033  -1.13783929  0.94524941]
 [ 2.91800865 -0.89635555 -2.53099566]] 
WHO
 [[1.87740393 1.85226722 1.93494814]]
[0. 0.] [0.00783044] [0]
[0. 1.] [0.99763604] [1]
[1. 0.] [0.96664966] [1]
[1. 1.] [0.01574311] [0]
Fehler:  128.97793862514519
